In [7]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from pyproj import Proj
import cartopy.feature as cfeature
from pathlib import Path
from matplotlib.colors import BoundaryNorm 
import glob
import cmaqsatproc as csp

In [12]:
# Define the file paths for BASE and ASSIM runs
base_files = glob.glob('/work/MOD3DEV/jkumm/EMBER/CMAQ/36US3/eval/POST/EMBER_BASE/*/AQS_Hourly_EMBER_BASE.csv')
base_files = [f for f in base_files if any(month in f for month in ['202305', '202306', '202307', '202308', '202309'])]
assim_files = glob.glob('/work/MOD3DEV/jkumm/EMBER/CMAQ/36US3/eval/POST/EMBER_ASSIM/*/AQS_Hourly_EMBER_ASSIM.csv')
assim_files = [f for f in assim_files if any(month in f for month in ['202305', '202306', '202307', '202308', '202309'])]
# scale_files = glob.glob('/work/MOD3DEV/jkumm/EMBER/CMAQ/36US3/eval/POST/EMBER_SCALE_ASSIM_i4/*/AQS_Hourly_EMBER_SCALE_ASSIM_i4.csv')
scale_assim_files = glob.glob('/work/MOD3DEV/jkumm/EMBER/CMAQ/36US3/eval/POST/EMBER_SCALE_ASSIM_FINAL_202506/*/AQS_Hourly_EMBER_SCALE_ASSIM_FINAL_202506.csv')

In [13]:
# Calculate metrics
def calculate_metrics(obs, mod):
    correlation = obs.corr(mod)
    rmse = np.sqrt(mean_squared_error(obs, mod))
    nmb = 100 * ((mod - obs).sum() / obs.sum())
    nme = 100 * (np.abs(mod - obs).sum() / obs.sum())
    return correlation, rmse, nmb, nme

In [14]:
# Initialize lists to store monthly statistics
monthly_stats_base = []
monthly_stats_assim = []
monthly_stats_scale_assim = []

# Loop through each month's files
for base_file, assim_file, scale_assim_file in zip(base_files, assim_files, scale_assim_files):

    # Read the data
    bdf = pd.read_csv(base_file, skiprows=5, na_values=[-999])
    adf = pd.read_csv(assim_file, skiprows=5, na_values=[-999])
    cdf = pd.read_csv(scale_assim_file, skiprows=5, na_values=[-999])
    
    # Rename 'O3_8hrmax_9cell_mod' column in cdf to 'O3_8hrmax_9cell_mod_scale_assim' for clarity
    cdf = cdf.rename(columns={'O3_8hrmax_9cell_mod': 'O3_8hrmax_9cell_mod_scale_assim'})
    
    # Reset indices to ensure alignment
    bdf = bdf.reset_index()
    adf = adf.reset_index()
    cdf = cdf.reset_index()

    # cdf = cdf.rename(columns={'O3_8hrmax_9cell_mod': 'O3_8hrmax_9cell_mod_scale_assim'})

    # Drop rows with NaN values in the relevant columns
    valid_data = bdf[['O3_8hrmax_9cell_ob', 'O3_8hrmax_9cell_mod']].join(adf[['O3_8hrmax_9cell_mod']], lsuffix='_base', rsuffix='_assim').dropna()

    valid_data = valid_data.join(cdf[['O3_8hrmax_9cell_mod_scale_assim']]).dropna()

    # Debug: Print intermediate data
    print(valid_data.head())

    # Observed and modeled O3_8hrmax_9cell for BASE and ASSIM
    obs_O3_8hrmax_9cell = valid_data['O3_8hrmax_9cell_ob']
    base_O3_8hrmax_9cell = valid_data['O3_8hrmax_9cell_mod_base']
    assim_O3_8hrmax_9cell = valid_data['O3_8hrmax_9cell_mod_assim']
    scale_assim_O3_8hrmax_9cell = valid_data['O3_8hrmax_9cell_mod_scale_assim']
    
    # Calculate metrics for BASE and ASSIM
    base_metrics = calculate_metrics(obs_O3_8hrmax_9cell, base_O3_8hrmax_9cell)
    assim_metrics = calculate_metrics(obs_O3_8hrmax_9cell, assim_O3_8hrmax_9cell)
    scale_assim_metrics = calculate_metrics(obs_O3_8hrmax_9cell, scale_assim_O3_8hrmax_9cell)

    # Append to the monthly statistics lists
    monthly_stats_base.append(base_metrics)
    monthly_stats_assim.append(assim_metrics)
    monthly_stats_scale_assim.append(scale_assim_metrics)

KeyError: "None of [Index(['O3_8hrmax_9cell_ob', 'O3_8hrmax_9cell_mod'], dtype='object')] are in the [columns]"

In [ ]:
def plot_aqs_exceedances(bdf, cdf):
    """
    Plots the number of MDA8 O3 exceedances (MDA8 O3 > 70 ppb) for each AQS dot
    on a Lambert Conformal projection map.

    Parameters:
        bdf (pd.DataFrame): DataFrame for BASE data with lat/lon and MDA8 O3 values.
        cdf (pd.DataFrame): DataFrame for SCALE ASSIM data with lat/lon and MDA8 O3 values.
    """
    import cartopy.crs as ccrs
    import cartopy.feature as cfeature

    # Count exceedances for BASE and SCALE ASSIM
    bdf['Exceedance'] = bdf['O3_8hrmax_9cell_mod'] > 70
    cdf['Exceedance'] = cdf['O3_8hrmax_9cell_mod_scale_assim'] > 70

    bdf_exceedances = bdf.groupby(['SiteId', 'Latitude', 'Longitude'])['Exceedance'].sum().reset_index()
    cdf_exceedances = cdf.groupby(['SiteId', 'Latitude', 'Longitude'])['Exceedance'].sum().reset_index()

    # Calculate SCALE_ASSIM - BASE exceedance difference
    diff_exceedances = cdf_exceedances.copy()
    diff_exceedances['Exceedance_Diff'] = cdf_exceedances['Exceedance'] - bdf_exceedances['Exceedance']

    # Set up the map projection
    projection = ccrs.LambertConformal(central_longitude=-97, central_latitude=40, standard_parallels=(33, 45))
    fig, axes = plt.subplots(3, 1, figsize=(16, 12), subplot_kw={'projection': projection})

    # Add map features
    for ax in axes:
        ax.add_feature(cfeature.COASTLINE, linewidth=1)
        ax.add_feature(cfeature.BORDERS, linewidth=0.5)
        ax.add_feature(cfeature.STATES, linewidth=0.5, linestyle='--')
        ax.set_extent([-125, -66.5, 24, 49], crs=ccrs.PlateCarree())

    # Plot BASE exceedances
    sc1 = axes[0].scatter(bdf_exceedances['Longitude'], bdf_exceedances['Latitude'], 
                          c=bdf_exceedances['Exceedance'], cmap='viridis', s=50, 
                          transform=ccrs.PlateCarree(), edgecolor='black')
    axes[0].set_title('BASE MDA8 O3 Exceedances', fontsize=14, weight='bold')
    plt.colorbar(sc1, ax=axes[0], orientation='vertical', label='Exceedances')

    # Plot SCALE ASSIM exceedances
    sc2 = axes[1].scatter(cdf_exceedances['Longitude'], cdf_exceedances['Latitude'], 
                          c=cdf_exceedances['Exceedance'], cmap='viridis', s=50, 
                          transform=ccrs.PlateCarree(), edgecolor='black')
    axes[1].set_title('SCALE ASSIM MDA8 O3 Exceedances', fontsize=14, weight='bold')
    plt.colorbar(sc2, ax=axes[1], orientation='vertical', label='Exceedances')

    # Plot exceedance difference (SCALE_ASSIM - BASE)
    sc3 = axes[2].scatter(diff_exceedances['Longitude'], diff_exceedances['Latitude'], 
                          c=diff_exceedances['Exceedance_Diff'], cmap='coolwarm', s=50, 
                          transform=ccrs.PlateCarree(), edgecolor='black')
    axes[2].set_title('Exceedance Difference (SCALE ASSIM - BASE)', fontsize=14, weight='bold')
    plt.colorbar(sc3, ax=axes[2], orientation='vertical', label='Difference')

    # Adjust layout
    plt.tight_layout()
    plt.savefig('AQS_MDA8_O3_Exceedances_Map_Publication.png', dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
    # Count exceedances for BASE and SCALE ASSIM
    bdf['Exceedance_base'] = bdf['O3_8hrmax_9cell_mod'] > 70
    cdf['Exceedance_scale_assim'] = cdf['O3_8hrmax_9cell_mod_scale_assim'] > 70
    bdf_exceedances = bdf.groupby(['SiteId', 'Latitude', 'Longitude'])['Exceedance_base'].sum().reset_index()
    cdf_exceedances = cdf.groupby(['SiteId', 'Latitude', 'Longitude'])['Exceedance_scale_assim'].sum().reset_index()

NameError: name 'bdf' is not defined